In [19]:
import random
import json

class PlanningTask:
    def __init__(self, num_search_spaces, num_aux_vars):
        self.num_search_spaces = num_search_spaces
        self.num_aux_vars = num_aux_vars
        self.num_vars = 3 + num_aux_vars
        self.num_operators = num_aux_vars * 2 + (num_aux_vars + 2) * num_search_spaces 
        self.var_id = dict()
        self.space = model = [random.randint(0,1) for i in range(num_search_spaces)]
        print("Search space: " + str(self.space))
        
    def __str__(self):
        s = self.get_version()
        s += self.get_metric()
        s += self.get_variables()
        s += "0\n"
        s += self.get_init_state()
        s += self.get_goal_states()
        s += self.get_operators()
        s += "0\n"
        return s
    
    def print_to_file(self, file_name):
        with open(file_name, "w") as text_file:
            text_file.write(str(self))
            
    def print_optimal_policy(self, file_name):
        optimal = [0] # inital state expansion
        for s in self.space:
            optimal += [s for i in range(num_aux_vars+2)]
        with open(file_name, "w") as outfile:
            json.dump(optimal, outfile)
    
    def get_var_id(self, name):
        return str(self.var_id[name])
    
    def get_version(self):
        s = "begin_version\n"
        s += "3\n"
        s += "end_version\n"
        return s
    
    def get_metric(self):
        s = "begin_metric\n"
        s += "1\n"
        s += "end_metric\n"
        return s

    def get_variables(self):
        s = str(self.num_vars) + "\n"
        
        # ss variable to track the search space
        self.var_id["ss"] = 0
        s += "begin_variable\n"
        s += "ss\n"
        s += "-1\n"
        s += str(self.num_search_spaces + 1) + "\n"
        for i in range(0, self.num_search_spaces + 1):
            s += "value-" + str(i) + "\n"
        s += "end_variable\n"

        # v0 incrementar to go trough search space
        self.var_id["v0"] = 1
        s += "begin_variable\n"
        s += "v0\n"
        s += "-1\n"
        s += str(2 + self.num_aux_vars) + "\n"
        for i in range(0, 2 + self.num_aux_vars):
            s += "value-" + str(i) + "\n"
        s += "end_variable\n"

        # v1 incrementar to go trough search space
        self.var_id["v1"] = 2
        s += "begin_variable\n"
        s += "v1\n"
        s += "-1\n"
        s += str(2 + self.num_aux_vars) + "\n"
        for i in range(0, 2 + self.num_aux_vars):
            s += "value-" + str(i) + "\n"
        s += "end_variable\n"

        # x variables as aux variables
        for x in range(0,self.num_aux_vars):
            self.var_id["x" + str(x)] = 3 + x
            s += "begin_variable\n"
            s += "x" + str(x) + "\n"
            s += "-1\n"
            s += "2\n"
            s += "value-0\n"
            s += "value-1\n"
            s += "end_variable\n"

        return s

    def get_mutexes(self):
        return "0\n"

    def get_init_state(self):
        s = "begin_state\n"
        for i in range(0,self.num_vars):
            s += "0\n"
        s += "end_state\n"
        return s
    
    def get_goal_states(self):
        s = "begin_goal\n"
        s += "1\n"
        s += self.get_var_id("ss") + " " + str(self.num_search_spaces) + "\n"
        s += "end_goal\n"
        return s
    
    def get_operators(self):
        s = str(self.num_operators) + "\n"
        for x in range(0, self.num_aux_vars):
            s += self.get_aux_set_operator("x" + str(x), 0)
            s += self.get_aux_set_operator("x" + str(x), 1)
        for i,x in enumerate(self.space):
            s += self.get_optimal_operators("v" + str(x), i) 
        return s
        
    def get_aux_set_operator(self, aux_name, value):
        s = "begin_operator\n"
        s += "set_" + aux_name + "_to_" + str(value) + "\n"
        s += "0\n"
        s += "1\n"
        s += "0 " + self.get_var_id(aux_name) + " -1 " + str(value) + "\n"
        s += "1\n"
        s += "end_operator\n"
        return s
    
    def get_optimal_operators(self, var_name, ss_cur_value):
        s = ""
        for i in range(0, self.num_aux_vars+1):
            s += "begin_operator\n"
            s += "inc_ss_" + str(ss_cur_value) + "_" + var_name + "_" + str(i) + "_" + str(i+1) + "\n"
            s += "1\n"
            s += self.get_var_id("ss") + " " + str(ss_cur_value) + "\n"
            s += "1\n"
            s += "0 " + self.get_var_id(var_name) + " " + str(i) + " " + str(i+1) + "\n"
            s += "1\n"
            s += "end_operator\n"
        s += "begin_operator\n"
        s += "inc_ss_" + str(ss_cur_value) + "_" + var_name + "_" + str(self.num_aux_vars+1) + "_" + str(self.num_aux_vars+2) + "\n"
        s += "0\n"
        s += "2\n"
        s += "0 " + self.get_var_id(var_name) + " " + str(self.num_aux_vars+1) + " 0\n"
        s += "0 " + self.get_var_id("ss") + " " + str(ss_cur_value) + " " + str(ss_cur_value+1) + "\n"
        s += "1\n"
        s += "end_operator\n"
        return s
    

In [22]:
import os

for i in range(30):
    folder = "instance" + str(i)
    os.mkdir(folder)
    num_search_space = random.randint(10,20)
    num_aux_vars = random.randint(10,20)
    task = PlanningTask(num_search_space, num_aux_vars)
    task.print_to_file(folder + "/instance.sas")
    task.print_optimal_policy(folder + "/optimal.json")

Search space: [1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]
Search space: [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1]
Search space: [1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1]
Search space: [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1]
Search space: [0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1]
Search space: [1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0]
Search space: [1, 0, 1, 0, 0, 0, 1, 0, 0, 0]
Search space: [1, 1, 1, 1, 0, 1, 0, 1, 0, 0]
Search space: [0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1]
Search space: [1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1]
Search space: [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]
Search space: [1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1]
Search space: [0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0]
Search space: [1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1]
Search space: [1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0]
Search space: [1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
Search space: [0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1]
Sear